In [1]:
username = 'meganorm-mznasrabadi'
datasets = {
    'BTNRH': {
        'base_dir': '/project/meganorm/Data/BTNRH/BIDS',
        'task': "task-rest",
        "ending" : "meg.fif"
    },
    'CAMCAN': {
        'base_dir': '/project/meganorm/Data/camcan/BIDS',
        'task': "task-rest",
        "ending" : "meg.fif"
    },
    'NIMH': {
        'base_dir': '/project/meganorm/Data/NIMH',
        'task': "task-rest",
        "ending" : "meg.ds"
    },
    'OMEGA': {
        'base_dir': '/project/meganorm/Data/Omega',
        'task': "task-rest",
        "ending" : "meg.ds"
    },
    'HCP': {
        'base_dir': '/project/meganorm/Data/HCP',
        'task': "",
        "ending" : "4-Restin/4D"
    }
    }

package_path = f'/home/{username}/MEGaNorm/'

In [2]:
import os
os.chdir(package_path)
from utils.parallel import submit_jobs, check_jobs_status, collect_results
from datasets.camcan import load_camcan_data
from datasets.btnrh import load_BTNRH_data
from utils.nm import hbr_data_split, estimate_centiles
from plots.plots import plot_nm_range_site, plot_comparison, plot_neurooscillochart, plot_age_dist2, plot_growthcharts
from utils.nm import model_quantile_evaluation, calculate_oscilochart
from utils.IO import merge_datasets_with_regex, merge_fidp_demo, merge_datasets_with_glob
import pandas as pd
import json
from pcntoolkit.normative_parallel import execute_nm
import warnings
warnings.filterwarnings("ignore")

In [3]:
def make_config(project, path=None):

    # preprocess configurations =================================================
    # downsample data
    config = dict()

    # You could also set layout to None to have high 
    # choices: all, lobe, None
    config["which_layout"] = "all"

    # which sensor type should be used
    # choices: meg, mag, grad, eeg, opm
    config["which_sensor"] = "meg"
    # config['fs'] = 1000

    # ICA configuration
    config['ica_n_component'] = 30
    config['ica_max_iter'] = 800
    config['ica_method'] = "fastica"

    # lower and upper cutoff frequencies in a bandpass filter
    config['cutoffFreqLow'] = 1
    config['cutoffFreqHigh'] = 45

    config["resampling_rate"] = 1000
    config["digital_filter"] = True
    config["notch_filter"] = False

    config["apply_ica"] = True

    config["auto_ica_corr_thr"] = 0.9

    # options are "average", "REST", and None 
    config["rereference_method"]= "average"
    
    # variance threshold across time
    config["mag_var_threshold"] = 4e-12
    config["grad_var_threshold"] = 4000e-13
    config["eeg_var_threshold"] = 40e-6
    # flatness threshold across time
    config["mag_flat_threshold"] = 10e-15
    config["grad_flat_threshold"] = 10e-15
    config["eeg_flat_threshold"] = 40e-6
    # variance thershold across channels
    config["zscore_std_thresh"] = 15 # change this

    # segmentation ==============================================
    #start time of the raw data to use in seconds, this is to avoid possible eye blinks in close-eyed resting state. 
    config['segments_tmin'] = 20
    # end time of the raw data to use in seconds, this is to avoid possible eye blinks in close-eyed resting state.
    config['segments_tmax'] = -20
    # length of MEG segments in seconds
    config['segments_length'] = 10
    # amount of overlap between MEG sigals in seconds
    config['segments_overlap'] = 2

    # PSD ==============================================
    # Spectral estimation method
    config['psd_method'] = "welch"
    # amount of overlap between windows in Welch's method
    config['psd_n_overlap'] = 1
    config['psd_n_fft'] = 2
    # number of samples in psd
    config["psd_n_per_seg"] = 2

    # fooof analysis configurations ==============================================
    # Desired frequency range to run FOOOF
    config['fooof_freqRangeLow'] = 3
    config['fooof_freqRangeHigh'] = 40
    # which mode should be used for fitting; choices (knee, fixed)
    config["aperiodic_mode"] = "knee"
    # minimum acceptable peak width in fooof analysis
    config["fooof_peak_width_limits"] = [1.0, 12.0]
    #Absolute threshold for detecting peaks
    config['fooof_min_peak_height'] = 0
    #Relative threshold for detecting peaks
    config['fooof_peak_threshold'] = 2

    # feature extraction ==========================================================
    # Define frequency bands
    config['freq_bands'] = {
                            'Theta': (3, 8),
                            'Alpha': (8, 13),
                            'Beta': (13, 30),
                            'Gamma': (30, 40),
                            # 'Broadband': (3, 40)
                            }

    # Define individualized frequency range over main peaks in each freq band
    config['individualized_band_ranges'] = { 
                                            'Theta': (-2, 3),
                                            'Alpha': (-2, 3), # change to (-4,2)
                                            'Beta': (-8, 9),
                                            'Gamma': (-5, 5)
                                            }

    # least acceptable R squred of fitted models
    config['min_r_squared'] = 0.9 
 
    config['feature_categories'] = {
                                    "Offset":False,
                                    "Exponent":False,
                                    "Peak_Center":False,
                                    "Peak_Power":False,
                                    "Peak_Width":False,
                                    "Adjusted_Canonical_Relative_Power":True, 
                                    "Adjusted_Canonical_Absolute_Power":False,
                                    "Adjusted_Individualized_Relative_Power":False,
                                    "Adjusted_Individualized_Absolute_Power":False,
                                    "OriginalPSD_Canonical_Relative_Power":True, 
                                    "OriginalPSD_Canonical_Absolute_Power":False,
                                    "OriginalPSD_Individualized_Relative_Power":False,
                                    "OriginalPSD_Individualized_Absolute_Power":False,
                                    }
    
    config["fooof_res_save_path"] = None

    config["random_state"] = 42

    if path is not None:
        out_file = open(os.path.join(path, project + ".json"), "w") 
        json.dump(config, out_file, indent = 6) 
        out_file.close()

    return config 

In [4]:
project = "natureArticle"
run = 0
project_dir = f'/home/{username}/Results/{project}/'

mainParallel_path = os.path.join(package_path, 'src', 'mainParallel.py')

features_dir = os.path.join(project_dir, 'Features')
features_log_path = os.path.join(features_dir, 'log')
features_temp_path = os.path.join(features_dir,'temp')

nm_processing_dir = os.path.join(project_dir, 'NM', 'Run_' + str(run))

job_configs = {'log_path':features_log_path, 'module':'mne', 'time':'1:00:00', 'memory':'20GB', 
                'partition':'normal', 'core':1, 'node':1, 'batch_file_name':'batch_job'}

if not os.path.isdir(features_log_path):
    os.makedirs(features_log_path)

if not os.path.isdir(features_temp_path):
    os.makedirs(features_temp_path)
    
if not os.path.isdir(nm_processing_dir):
    os.makedirs(nm_processing_dir)
    
configs = make_config(project, project_dir)

subjects = merge_datasets_with_glob(datasets)

In [5]:
# subjects = {"sub-0240" : subjects["sub-0240"]}

In [6]:
### Parallel feature extraction  

# # Running Jobs
start_time = submit_jobs(mainParallel_path, features_dir, subjects, 
                features_temp_path, job_configs=job_configs, config_file=os.path.join(project_dir, project+'.json'))
# Checking jobs
failed_jobs = check_jobs_status(username, start_time)

falied_subjects = {failed_job:subjects[failed_job] for failed_job in failed_jobs}

while len(failed_jobs)>0:
    # Re-running Jobs
    start_time = submit_jobs(mainParallel_path, features_dir, falied_subjects, 
                features_temp_path, job_configs=job_configs, config_file=os.path.join(project_dir, project+'.json'))
    # Checking jobs
    failed_jobs = check_jobs_status(username, start_time)

collect_results(features_dir, subjects, features_temp_path, file_name='all_features', clean=False)

Submitted batch job 15008657
Submitted batch job 15008658
Submitted batch job 15008659
Submitted batch job 15008660
Submitted batch job 15008661
Submitted batch job 15008662
Submitted batch job 15008663
Submitted batch job 15008664
Submitted batch job 15008665
Submitted batch job 15008666
Submitted batch job 15008667
Submitted batch job 15008668
Submitted batch job 15008669
Submitted batch job 15008670
Submitted batch job 15008671
Submitted batch job 15008672
Submitted batch job 15008673
Submitted batch job 15008674
Submitted batch job 15008675
Submitted batch job 15008676
Submitted batch job 15008677
Submitted batch job 15008678
Submitted batch job 15008679
Submitted batch job 15008680
Submitted batch job 15008681
Submitted batch job 15008683
Submitted batch job 15008684
Submitted batch job 15008685
Submitted batch job 15008686
Submitted batch job 15008687
Submitted batch job 15008688
Submitted batch job 15008689
Submitted batch job 15008690
Submitted batch job 15008691
Submitted batc

In [ ]:
### Data preparation for Normative Modeling
base_dirs = [values["base_dir"] for values in datasets.values()]
dataset_names = list(datasets.keys())
merged_data = merge_fidp_demo(base_dirs, features_dir, dataset_names, include_patients=False)

biomarker_num = hbr_data_split(merged_data, nm_processing_dir, drop_nans=True, batch_effects=['sex', 'site', "eyes"])

In [ ]:
### Setting up NM configs

python_path = '/project/meganorm/Software/Miniconda3/envs/mne/bin/python' 

hbr_configs = {
                'homo_Gaussian_linear':{'model_type':'linear', 'likelihood':'Normal', 'linear_sigma':'False',
                                        'random_slope_mu':'False', 'linear_epsilon':'False', 'linear_delta':'False'}, 
                'homo_Gaussian_bspline':{'model_type':'bspline', 'likelihood':'Normal', 'linear_sigma':'False',
                                        'random_slope_mu':'False', 'linear_epsilon':'False', 'linear_delta':'False'}, 
                'homo_SHASH_linear':{'model_type':'linear', 'likelihood':'SHASHb', 'linear_sigma':'False',
                                    'random_slope_mu':'False', 'linear_epsilon':'False', 'linear_delta':'False'}, 
                'homo_SHASH_bspline':{'model_type':'bspline', 'likelihood':'SHASHb', 'linear_sigma':'False',
                                    'random_slope_mu':'False', 'linear_epsilon':'False', 'linear_delta':'False'}, 
                'hetero_Gaussian_linear':{'model_type':'linear', 'likelihood':'Normal', 'linear_sigma':'True',
                                        'random_slope_mu':'False', 'linear_epsilon':'False', 'linear_delta':'False'},
                'hetero_Gaussian_bspline':{'model_type':'bspline', 'likelihood':'Normal', 'linear_sigma':'True',
                                        'random_slope_mu':'False', 'linear_epsilon':'False', 'linear_delta':'False'},
                'hetero_SHASH_linear':{'model_type':'linear', 'likelihood':'SHASHb', 'linear_sigma':'True',
                                    'random_slope_mu':'False', 'linear_epsilon':'True', 'linear_delta':'True'},
                'hetero_SHASH_bspline':{'model_type':'bspline', 'likelihood':'SHASHb', 'linear_sigma':'True',
                                        'random_slope_mu':'False', 'linear_epsilon':'True', 'linear_delta':'True'},
            }

inscaler='None' 
outscaler='minmax' 
batch_size = 1
outputsuffix = '_estimate'

respfile = os.path.join(nm_processing_dir, 'y_train.pkl')
covfile = os.path.join(nm_processing_dir, 'x_train.pkl')

testrespfile_path = os.path.join(nm_processing_dir, 'y_test.pkl')
testcovfile_path = os.path.join(nm_processing_dir, 'x_test.pkl')

trbefile = os.path.join(nm_processing_dir, 'b_train.pkl')
tsbefile = os.path.join(nm_processing_dir, 'b_test.pkl')

memory = '2gb'
duration = '5:00:00'
cluster_spec = 'slurm'

In [ ]:
### Running NM

#for method in hbr_configs.keys():
method = 'hetero_SHASH_bspline'
processing_dir = os.path.join(nm_processing_dir, method) + '/'
nm_log_path = os.path.join(processing_dir, 'log') + '/'

if not os.path.isdir(processing_dir):
    os.makedirs(processing_dir)
if not os.path.isdir(nm_log_path):
    os.makedirs(nm_log_path)

execute_nm(processing_dir, python_path,
            'NM', covfile, respfile, batch_size, memory, duration, alg='hbr', 
            log_path=nm_log_path, binary=True, testcovfile_path=testcovfile_path, 
            testrespfile_path=testrespfile_path,trbefile=trbefile, tsbefile=tsbefile, 
            model_type=hbr_configs[method]['model_type'], likelihood=hbr_configs[method]['likelihood'],  
            linear_sigma=hbr_configs[method]['linear_sigma'], random_slope_mu=hbr_configs[method]['random_slope_mu'],
            linear_epsilon=hbr_configs[method]['linear_epsilon'], linear_delta=hbr_configs[method]['linear_delta'], 
            savemodel='True', inscaler=inscaler, outscaler=outscaler, outputsuffix=outputsuffix, 
            interactive='auto', cluster_spec=cluster_spec)

In [ ]:
### Evaluating quantiles using MACE

mace, best_models, bio_ids = model_quantile_evaluation(hbr_configs, nm_processing_dir, testcovfile_path, 
                              testrespfile_path, tsbefile, biomarker_num, plot=False, outputsuffix='estimate')


plot_comparison(nm_processing_dir, hbr_configs, biomarker_num)

In [ ]:
## Plotting ranges
for config in hbr_configs.keys():
    processing_path = os.path.join(nm_processing_dir, config)
    
    q = estimate_centiles(processing_path, biomarker_num, quantiles=[0.05, 0.25, 0.5, 0.75, 0.95],
                          batch_map={0:{'Male':0, 'Female':1}, 1:{'CAMCAN':0,'BTNRH':1}}, 
                          age_range=[6, 80])
    plot_nm_range_site(processing_path, nm_processing_dir, experiment_id=2)

# exp 0: ap - periodic > 0 => 0 and hetero_SHASH_bspline
# exp 1: hetero_SHASH_bspline
# exp 2:  ap - periodic > 0 => 0 and hetero_SHASH_bspline and -inf are removed 

In [ ]:
# plot age distribution for different sites and train/test/validation partitions
base_dir = "/home/meganorm-mznasrabadi/Results/BIOMAG2024/NM"
plot_age_dist2(base_dir, val=False)

In [ ]:
## Fitting on full data

feature_path = os.path.join(features_dir, "all_features.csv")
BTNRH_cov_path = "/project/meganorm/Data/BTNRH/BTNRH/BIDS_data/participants.tsv"
camcan_cov_path = '/project/meganorm/Data/camcan/CamCAN/cc700/participants.tsv'

camcan_data = load_camcan_data(feature_path, camcan_cov_path)
BTNRH_data = load_BTNRH_data(feature_path, BTNRH_cov_path)

merged_data = pd.concat([BTNRH_data, camcan_data], axis=0)

merged_data['age'] = merged_data['age']/100

biomarker_num = hbr_data_split(merged_data, nm_processing_dir, drop_nans=True, batch_effects=['gender', 'site'], 
                               train_split=0.99, prefix='final_')

python_path = '/project/meganorm/Software/Miniconda3/envs/mne/bin/python' 

hbr_configs = {
                'final':{'model_type':'bspline', 'likelihood':'SHASHb', 'linear_sigma':'True',
                                        'random_slope_mu':'False', 'linear_epsilon':'True', 'linear_delta':'True'},
                }

inscaler='None' 
outscaler='minmax' 
batch_size = 1
outputsuffix = '_estimate'

respfile = os.path.join(nm_processing_dir, 'final_y_train.pkl')
covfile = os.path.join(nm_processing_dir, 'final_x_train.pkl')

testrespfile_path = os.path.join(nm_processing_dir, 'final_y_test.pkl')
testcovfile_path = os.path.join(nm_processing_dir, 'final_x_test.pkl')

trbefile = os.path.join(nm_processing_dir, 'final_b_train.pkl')
tsbefile = os.path.join(nm_processing_dir, 'final_b_test.pkl')

memory = '4gb'
duration = '5:00:00'
cluster_spec = 'slurm'

method = 'final'
processing_dir = os.path.join(nm_processing_dir, method) + '/'
nm_log_path = os.path.join(processing_dir, 'log') + '/'

if not os.path.isdir(processing_dir):
    os.makedirs(processing_dir)
if not os.path.isdir(nm_log_path):
    os.makedirs(nm_log_path)

execute_nm(processing_dir, python_path,
            'NM', covfile, respfile, batch_size, memory, duration, alg='hbr', 
            log_path=nm_log_path, binary=True, testcovfile_path=testcovfile_path, 
            testrespfile_path=testrespfile_path,trbefile=trbefile, tsbefile=tsbefile, 
            model_type=hbr_configs[method]['model_type'], likelihood=hbr_configs[method]['likelihood'],  
            linear_sigma=hbr_configs[method]['linear_sigma'], random_slope_mu=hbr_configs[method]['random_slope_mu'],
            linear_epsilon=hbr_configs[method]['linear_epsilon'], linear_delta=hbr_configs[method]['linear_delta'], 
            savemodel='True', inscaler=inscaler, outscaler=outscaler, outputsuffix=outputsuffix, 
            interactive='auto', cluster_spec=cluster_spec)

q = estimate_centiles(processing_dir, biomarker_num, quantiles=[0.05, 0.25, 0.5, 0.75, 0.95],
                        batch_map={0:{'Male':0, 'Female':1}, 1:{'CAMCAN':0,'BTNRH':1}}, 
                        age_range=[6, 80])
plot_nm_range_site(processing_dir, nm_processing_dir, experiment_id=2)

#plot_growthcharts(processing_dir, idp_indices=[9,6,7,8], 
#                  idp_names=['Relative Adjusted Theta', 'Relative Adjusted Alpha', 
#                             'Relative Adjusted Beta', 'Relative Adjusted Gamma'])
plot_growthcharts(processing_dir, idp_indices=list(range(39)), 
                  idp_names=['Absolute Adjusted Alpha', 'Absolute Adjusted Beta', 'Absolute Adjusted Total Power', 'Absolute Adjusted Gamma', 
                             'Absolute Adjusted Theta', 'Absolute Adjusted Theta-Beta Ratio',
                             'Relative Adjusted Alpha', 'Relative Adjusted Beta', 'Relative Adjusted Gamma', 'Relative Adjusted Theta',  
                             'Absolute Adjusted Individualized Alpha', 'Absolute Adjusted Individualized Beta',
                             'Relative Adjusted Individualized Alpha', 'Relative Adjusted Individualized Beta', 'Exponent', 'Offset',
                             'Absolute Alpha Power', 'Absolute Beta Power', 'Absolute Total Power', 'Absolute Gamma Power', 'Absolute Theta Power',
                             'Relative Alpha Power', 'Relative Beta Power', 'Relative Gamma Power', 'Relative Theta Power',
                             'Absolute Individualized Alpha', 'Absolute Individualized Beta', 'Relative Absolute Individualized Alpha', 'Relative Absolute Individualized Beta',
                             'Peak Alpha', 'Peak Beta', 'Peak Broadband', 'Peak Power Alpha', 'Peak Power Beta', 'Peak Power Broadband',
                             'Peak Width Alpha', 'Peak Width Beta', 'Peak Width Broadband', 'Absolute Theta-Beta Ratio'])

In [ ]:
# Calculateing Oscilograms

gender_ids = {'Male':0, 'Female':1}
frequency_band_model_ids = {'Theta':9, 'Alpha':6, 'Beta':7, 'Gamma':8}
quantiles_path= os.path.join(processing_dir, 'Quantiles_estimate.pkl')
oscilograms, age_slices = calculate_oscilochart(quantiles_path, gender_ids, frequency_band_model_ids)

plot_neurooscillochart(oscilograms, age_slices, processing_dir)